<a href="https://colab.research.google.com/github/cabicho/ds_in_deploy/blob/modulo08_hyperparameter_fine_tuning/m04_v01_h6_store_sales_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalations

In [7]:
!pip install sklearn
!pip install inflection==0.5.1
#!pip install import-ipynb==0.1.4
!pip install boruta

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 1.8 MB/s eta 0:00:00


In [8]:
!pip freeze | grep sklearn #sklearn-pandas==2.2.0
#!pip freeze | gre

sklearn-pandas==2.2.0


In [9]:
!pip freeze | grep inflec #inflect==7.0.0 inflection==0.5.1
!pip freeze | grep import-ipynb
!pip freeze | grep import-ipynb

inflect==7.0.0
inflection==0.5.1


# 0.0 IMPORTS

In [10]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import pandas as pd
#!pip install inflection
import inflection
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#plt.style.use('seaborn-white')
import datetime
#import import-ipynb #import-ipynb

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# import matplotlib.gridspec # as gridspec
# H4. Lojas com promoções activas por mais tempo, com + dias de promoção, deveriam vender + *
from  matplotlib.gridspec   import GridSpec # as gridspec, ModuleNotFoundError: No module named 'matplotlib.gridspec.GridSpec'; 'matplotlib.gridspec' is not a package

from IPython.core.display   import HTML
from IPython.display        import Image
#%matplotlib inline
from boruta                 import BorutaPy

from sklearn.preprocessing  import RobustScaler, MinMaxScaler, LabelEncoder #trocando letras por números
from sklearn.ensemble       import RandomForestRegressor

from sklearn.metrics        import mean_absolute_error, mean_squared_error
from sklearn.linear_model   import LinearRegression, Lasso


# 0.1 Helper Functions

In [ ]:
#import-ipynb


## 0.1 Helper Functions

In [ ]:
def jupyter_settings():
    %matplotlib inline
    #%pylab inline

    plt.style.use('bmh')
    plt.rcParams['figure.figsize']=[25,12]
    plt.rcParams['font.size']=24

    display(HTML('<style>.container {width:100% !important;}</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option('display.expand_frame_repr', False)

    sns.set()

#4.1.3. Categorical Variable
def plots(df, n):
    #for i to n:
    #        plt.subplot(1,n,i+1)
    plt.subplot(1,n,1)
    sns.countplot(df5['state_holiday'], orient='h' ) # “y” | “h” |“v” |  “x” | “y”

    #plt.subplot(1,n,i+1)
    plt.subplot(1,n,2)
    #label='public holiday'
    #sns.kdeplot(df4[df4['state_holiday']==label]['sales'], label=label, shade=True) #'Easter holiday, Christmas
    # `shade` is now deprecated in favor of `fill`; setting `fill=True`.
    #This will become an error in seaborn v0.14.0; please update your code.
    #for i in
    #import numpy.dtype
    for i in range(len(df4.state_holiday.unique())): # identificando o tamanho da lista
        if df4.state_holiday.unique()[i]!='regular_day': # verificando o valor do indice
            #print(df4.state_holiday.unique()[i]) #visualizando o valor
            label=df4.state_holiday.unique()[i]
            #df4.state_holiday.unique()[i]
            #sns.kdeplot(df4[df==label]['sales'], label=label, fill=True) #'Easter holiday, Christmas
            #sns.kdeplot(df4[df==int(df4.state_holiday.unique()[i])]['sales'], label=label, fill=True)

            #np.dtype(df4.state_holiday.unique()[i]) # TypeError: data type 'public holiday' not understood
            #label=df4.state_holiday.unique()[i] #'public holiday'
            sns.kdeplot(df4[df4['state_holiday']==label]['sales'], label=label, fill=True)
            print(label)
    #o grafico sobreposto e transparente facilita a observação do pico e largura na distribuição das variáveis
    # visualizando as distribuições sobre postas

# 7.0. PASSO 07 - MACHINE LEARNING MODELLING
def mean_absolute_percentage_error(y, yhat):
  return np.mean(np.abs((y-yhat)/y))

def ml_error(model_name, y, yhat):
  mae= mean_absolute_error(y, yhat)
  mape= mean_absolute_percentage_error(y, yhat)
  rmse= np.sqrt(mean_squared_error(y, yhat))

  return pd.DataFrame({'Model Name' : model_name,
                       'MAE'        : mae,
                       'MAPE'       : mape,
                       'RMSE'       : rmse },
                      index=[0])

# Modulo 7, - MACHINE LEARNING MODELLING
def cross_validation(x_training, kfold, model_name, model): #(x_training, kfold): #try_go():

  day_=x_training[['date','store']].groupby('store').max().reset_index()['date'][0]
  print('Date in database: {}, periodo of time: {}'.format(day_, datetime.timedelta(days=6*7)))

  mae_list, mape_list, rmse_list = [],[],[]

  #filtering dataset
  for k in range(1,kfold+1): #6): # reversed(range(1,6)):
    #max_date_train_or_start_validation
    validation_start_date=day-datetime.timedelta(days=k*6*7)
    validation_end_date=day-datetime.timedelta(days=(k-1)*6*7)

    #validation_start_date=max_date_train_
    max_date_train_ = day-datetime.timedelta(days=k*6*7)
    #max_date_train_ = day-datetime.timedelta(days=k*6*7)
    print(max_date_train_) #[0] # = min_date_test Timestamp('2012-11-20 00:00:00')

    # filtering/training dataset
    training = x_training[x_training['date']< validation_start_date]#[cols_selected_boruta] #max_date_train_]
    validation = x_training[(x_training['date']>= validation_start_date)&(x_training['date']< validation_end_date)]#[cols_selected_boruta]
    # df6_train_ = df6[df6['date']< max_date_train_] #df6['date']< '2015-06-05'] #max_date_train
    # df6[df6_train].sample()
    # print(df6_train_.sample(1)) #608130, 969198

    #training and validation dataset

    #training dataset
    xtraining = training.drop(['date','sales'], axis=1) #removing colluns that can be into to the training
    ytraining = training.sales

    #print('xtraining.sample()', xtraining.sample())

    #validation dataset
    xvalidation = validation.drop(['date','sales'], axis=1) #removing colluns that can be into to the training
    yvalidation = validation.sales

    #print('xvalidation.sample()', xvalidation.sample())
    #X_train_ = df6_train_.copy()
    #y_train_ = X_train_['sales']

    #model
    #print(xtraining.dtypes)
    #model='Linear Regression'
    #lr = LinearRegression().fit(xtraining, ytraining)
    #
    m = model.fit(xtraining, ytraining)

    #prediction
    #yhat_lr=lr.predict(xvalidation)
    yhat=m.predict(xvalidation)

    #performance, model_name, 'Linear Regression'
    #lr_result = ml_error(model_name, np.expm1(yvalidation), np.expm1(yhat_lr))
    m_result = ml_error(model_name, np.expm1(yvalidation), np.expm1(yhat))
    #print(lr_result)
    print(m_result)

    #mae_list.append(lr_result['MAE'])
    mae_list.append(m_result['MAE'])
    #mape_list.append(lr_result['MAPE'])
    mape_list.append(m_result['MAPE'])
    #rmse_list.append(lr_result['RMSE'])
    rmse_list.append(m_result['RMSE'])

    #print('Train_ Min Date: {}'.format(xvalidation['date'].min()['date']))
    #print('Train_ Max Date: {}'.format(xvalidation.date.max()['date']))
  return pd.DataFrame(
      { 'Model Name': model_name,
        'MAE CV':  np.round( np.mean(mae_list), 2).astype(str) + ' +/- ' + np.round( np.std(mae_list), 2).astype(str),
        'MAE_CV':  ' [ ' + np.round( np.mean(mae_list) - np.std(mae_list), 2).astype(str) + ' -- ' + np.round( np.mean(mae_list) + np.std(mae_list), 2).astype(str) + ' ] ',
        'MAPE CV':  np.round( np.mean(mape_list), 2).astype(str) + ' +/- ' + np.round( np.std(mape_list), 2).astype(str),
        'MAPE_CV':  ' [ ' + np.round( np.mean(mape_list) - np.std(mape_list), 2).astype(str) + ' -- ' + np.round( np.mean(mape_list) + np.std(mape_list), 2).astype(str) + ' ] ',
        'RMSE CV':  np.round( np.mean(rmse_list), 2).astype(str) + ' +/- ' + np.round( np.std(rmse_list), 2).astype(str),
        'RMSE_CV': ' [ ' + np.round( np.mean(rmse_list) - np.std(rmse_list), 2).astype(str) + ' -- ' + np.round( np.mean(rmse_list) + np.std(rmse_list), 2).astype(str) + ' ] '},
       index=[0])

In [ ]:
jupyter_settings()

# 0.2 Loading Data

In [ ]:
#df_train_raw=pd.read_csv("../ds/data_set/data/train.csv", low_memory=False)
#df_train_raw=pd.read_csv("/dataset/data/train.csv", low_memory=False)
df_train_raw=pd.read_csv("/content/drive/MyDrive/dataset/train.csv", low_memory=False)

#df_store_raw=pd.read_csv("../ds/data_set/data/store.csv", low_memory=False)
df_store_raw=pd.read_csv("/content/drive/MyDrive/dataset/store.csv", low_memory=False)


In [ ]:
df_train_raw.sample()

In [ ]:
df_store_raw.sample()

In [ ]:
df_train_raw.shape #(1001599, 10)
df_train_raw.sample()

In [ ]:
df_store_raw.shape #(1115, 10)
df_store_raw.sample()

In [ ]:
#df_sales_raw # 1001599 rows × 10 columns
df_store_raw # 1115 rows × 10 columns

# arquivo_referencia, anexado a referencia, como faremos o merge, a coluna igual nos dois
df_raw=pd.merge(df_train_raw, df_store_raw, how='left', on='Store') #1001599 rows × 19 columns

# so alterou o numero de colunas
df_raw.shape #(1001599, 19)

In [ ]:
df_raw.sample()

# 1.0. PASSO 01 - DESCRIÇÃO DOS DADOS

## 1.1 Rename Columns

In [ ]:
# fazendo uma copia da seccao anterior
df1=df_raw.copy()
#list(df1.columns)

In [ ]:
cols_old=list(df1.columns)
cols_old #.list()

In [ ]:
snakecase = lambda x: inflection.underscore(x)
#mapeando a função
cols_new = list(map(snakecase, cols_old))
#rename
df1.columns=cols_new
#cols_new

## 1.2 Data Dimensions

In [ ]:
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Cols: {}'.format(df1.shape[1]))

## 1.3 Data Types

In [ ]:
df1.dtypes
df1['date']=pd.to_datetime(df1['date'])
#df1.dtypes

## 1.4 Check NA

In [ ]:
df1.isna().sum()

## 1.5 Fillout NA

In [ ]:
df1.sample().T

In [ ]:
atributo='competition_open_since_month'
df1[atributo]= df1.apply(lambda x: x['date'].month if math.isnan(x[atributo]) else x[atributo], axis=1) #axis=1 aplicando ao longo das colunas

In [ ]:
atributo='competition_open_since_year'
df1[atributo]= df1.apply(lambda x: x['date'].year if math.isnan(x[atributo]) else x[atributo], axis=1) #axis=1 aplicando ao longo das colunas

In [ ]:
atributo='promo2_since_week'
df1[atributo]= df1.apply(lambda x: x['date'].week if math.isnan(x[atributo]) else x[atributo], axis=1) #axis=1 aplicando ao longo das colunas

In [ ]:
atributo='promo2_since_year'
df1[atributo]= df1.apply(lambda x: x['date'].year if math.isnan(x[atributo]) else x[atributo], axis=1) #axis=1 aplicando ao longo das colunas

In [ ]:
#competition_distance              2600
#print(df1[1])
#df1['competition_distance'] =df1['competition_distance'].apply(lambda x: 200000.0 if math.isnan(x) else x)

#competition_open_since_month    318392
# se for verdade extraimos o mes
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].moth if math.isnan( x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)
print('done')
df1.sample(10)
#competition_open_since_year     318392
#atributo='competition_open_since_year'
#df1[atributo] = df1.apply(lambda x: x['date'].year if math.isnan( x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

#promo2_since_week               500415
#promo2_since_year               500415
#promo_interval                  500415

In [ ]:
#promo_interval                  500415
month_map = {1:'Jan', 2:'Fev', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
df1['promo_interval'].fillna(0, inplace=True) # fazendo modificação directa na coluna
#criando a coluna month_map, que indica o mês que ocorreu a venda
df1['month_map'] = df1['date'].dt.month.map(month_map) # os numeros serão trocados pelas letras
# A venda foi realizada num mês promocional
df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)
# criando lista dos elementos da coluna promo_interval e verificar se month_map está dentro da lista do intervalo

In [ ]:
df1[['date', 'month_map']].sample(12)

## 1.6 Check Type

In [ ]:
df1.dtypes

In [ ]:

#competition_open_since_month           float64
atributo='competition_open_since_month'
df1[atributo]=df1[atributo].astype(int)
#competition_open_since_year            float64
atributo='competition_open_since_year'
df1[atributo]=df1[atributo].astype(int)
#promo2_since_week                      float64
atributo='promo2_since_week'
df1[atributo]=df1[atributo].astype(int)
#promo2_since_year                      float64
atributo='promo2_since_year'
df1[atributo]=df1[atributo].astype(int)

## 1.7 Descriptive Statistical, ganhando conhecimento do negocio e detectar alguns erros

In [ ]:
num_attributes = df1.select_dtypes(include=['int64', 'float64']) # todos os atributos sao de variavel numerica
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]']) #include=['object']
obj_attributes = df1.select_dtypes(include=['object']) #include=['object']

In [ ]:
#cat_attributes.sample(2)
#if (cat_attributes == obj_attributes):
#  print('True')
#else:
#  print('False')

In [ ]:
obj_attributes.sample(2)

In [ ]:
num_attributes.sample() #

In [ ]:
# Central tendency - mean, median
# Dispersion - std, min, max, range, skew, kurtosis

### 1.7.1 Numerical Attributes

In [ ]:
# Central tendency - mean, median
ct1 = pd.DataFrame( num_attributes.apply(np.mean) ).T
ct2 = pd.DataFrame( num_attributes.apply(np.median) ).T

# Dispersion - std, min, max, range, skew, kurtosis
d1=pd.DataFrame(num_attributes.apply(np.std)).T
d2=pd.DataFrame(num_attributes.apply(min)).T
d3=pd.DataFrame(num_attributes.apply(max)).T
d4=pd.DataFrame(num_attributes.apply(lambda x: x.max()-x.min())).T
d5=pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6=pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

# Concatenate
m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
#nomeando os atributos, as colunas
m.columns=['attributes','min','max','range','mean','median','std','skew','kurtosis']
m

In [ ]:
#sns.distplot(df1['sales']) #`distplot` is a deprecated function and will be removed in seaborn v0.14.0.
sns.displot(df1['sales'])
sns.displot(df1['competition_distance'])
!pip freeze | grep seaborn

### 1.7.2 Categorial Attributes

In [ ]:
#Quantos niveis cada variável categorica tem? Como aplicar 1 função em todas as colunas?
cat_attributes.apply(lambda x: x.unique().shape[0])

In [ ]:
#Quantos niveis cada variável categorica tem? Como aplicar 1 função em todas as colunas?
cat_attributes.apply(lambda x: x.unique())

In [ ]:
# a dimensão das variáveis está muito diferente
# provavel razão, dias fechado tendo vendas igual a zero

In [ ]:
aux1=df1[(df1['state_holiday']!=0) & (df1['sales']>0)]
sns.boxplot(x='state_holiday', y='sales', data=df1)

In [ ]:
aux1=df1[(df1['state_holiday']!='0') & (df1['sales']>0) & (df1['sales']<25000)] #dia diferente do feriado e com vendas acima de zero

plt.subplot(1,7,1) #1 linha, 3 colunas, na posicão 1
sns.boxplot(x='state_holiday', y='sales', data=aux1)

plt.subplot(1,7,4) #1 linha, 3 colunas, na posicão 2
sns.boxplot(x='store_type', y='sales', data=aux1)

plt.subplot(1,7,7) #1 linha, 3 colunas, na posicão 1
sns.boxplot(x='assortment', y='sales', data=aux1)


# 2.0. PASSO 02 - FEATURE ENGINEERING

In [ ]:
df2=df1.copy()

## 2.1. Mapa Mental de Hipoteses

In [ ]:
Image('img/MindMapHipoteses_m03.png')

## 2.1. Criação de Hipoteses

### 2.1.1. Hipóteses Loja

** 1.** Lojas com maior quadro de funcionários deveriam vender mais.

** 1. ** Lojas com maior estoque deveriam vender mais.

**1.** Lojas com porte maior deveriam vender mais.

**1.** Lojas com menor porte deveriam vender menos.

**1.** Lojas com maior sortimento deveriam vender mais.

### 2.1.2. Hipóteses Produto

**1.** Lojas que investem mais em Marketing deveriam vender mais.

**2.** Lojas com maior exposição de produtos nas vitrines deveriam vender mais.

**3.** Lojas com produtos com preços menores deveriam vender mais.

**4.** Lojas com promoções mais agressivas (descontos maiores), deveriam vender mais.

**5.** Lojas com promoções activas por mais tempo deveriam vender mais.

**6.** Lojas com + dias de promoção deveriam vender +.

**4.** Lojas que tem preços menores por mais tempo nos produtos deveriam vender mais.

**5.** Lojas com + promoções consecutivas deveriam vender +.

### 2.1.3. Hipóteses Tempo

**1.** Lojas abertas durante o feriado de Natal deveriam vender menos.

**1.** Lojas ao longo do ano deveriam vender mais.

**1.** Lojas no segundo semestre do ano deveriam vender mais.

**1.** Lojas depois do dia 10 de cada mes deveriam vender mais.

**1.** Lojas nos finais de semana deveriam vender menos.

**1.** Lojas nos Feriados escolares deveriam vender menos.
    

## 2.2. Lista Final de Hipóteses

**[1., 3.]** Lojas com [maior sortimento/ competidores à + tempo]  deveriam vender +.

**[2., ]** Lojas com competidores + próximo deveriam vender - .

** Produtos **

**[4., 5., 6.]** Lojas [promoções activas por mais tempo, com + [dias de promoção, promoções consecutivas]]  deveriam vender +

** Tempo **

**7.** Lojas abertas durante o feriado de Natal deveriam vender menos.

**8.** Lojas ao longo do ano deveriam vender mais.

**9.** Lojas no segundo semestre do ano deveriam vender mais.

**10.** Lojas depois do dia 10 de cada mes deveriam vender mais.

**11.** Lojas nos finais de semana deveriam vender menos.

**12.** Lojas nos Feriados escolares deveriam vender menos.

## 2.3. Feature Engineeringn

In [ ]:
df2.sample()

In [ ]:
# year
df2['year']=df2['date'].dt.year

# month
df2['month']=df2['date'].dt.month

# day
df2['day']=df2['date'].dt.day

# week of year
# df2['week_of_year']=df2['date'].dt.weekofyear #AttributeError: 'DatetimeProperties' object has no attribute 'weekofyear'
# Deprecated since version 1.1.0.
#https://pandas.pydata.org/pandas-docs/version/1.3/reference/api/pandas.Series.dt.weekofyear.html

#df2['week_of_year']=df2['date'].dt.isocalendar()
#df2['week_of_year']=df2['date'].dt.isocalendar().week

#df2['week_of_year']=df2['date'].dt.week #AttributeError: 'DatetimeProperties' object has no attribute 'week'

# year week
df2['year_week']=df2['date'].dt.strftime('%Y-%W') #AttributeError: 'DatetimeProperties' object has no attribute 'srtftime'
df2['year_week'].sample(5)
# competition since
#competition_open_since_month	competition_open_since_year
year='competition_open_since_year'
month='competition_open_since_month'
# considerou-se que o primeiro dia do mes, o dia que iniciou a competição #tempo desde que a competição começou em meses
df2['competition_open_since']=df2.apply( lambda x: datetime.datetime(year=x[year],month=x[month],day=1), axis=1)
df2['competition_time_month']=((df2['date']-df2['competition_open_since'])/30).apply(lambda x: x.days).astype(int)
df2['competition_time_month'].sample()

# promo since
# promo2_since_week	promo2_since_year

# assortment
# state holiday

In [ ]:
# promo since
# promo2_since_week	promo2_since_year
df2['promo_since'] = df2['promo2_since_year'].astype(str) + '-' + df2['promo2_since_week'].astype(str)
df2['promo_since'].sample()


In [ ]:
#passando a campo 'promo_since' de string para datetime64
df2['promo_since']=df2['promo_since'].apply(lambda x: datetime.datetime.strptime(x + '-1', '%Y-%W-%w') - datetime.timedelta(days=7))

df2['promo_since'].sample()

In [ ]:
# calculando o tempo, em semanas, desde que a promoção está activa, entre duas datas
# sinal negativo, vendas no período tradicional da promoção
# sinal positivo, vendas no período extendido da promoção
df2['promo_time_week']=((df2['date']-df2['promo_since'])/7).apply(lambda x: x.days).astype(int)
df2['promo_time_week'].sample(5)

In [ ]:
# assortment

df2['assortment']=df2['assortment'].apply(lambda x: 'basic' if x == 'a' else 'extra' if x=='b' else 'extended')
df2['assortment'].sample(5)


In [ ]:
df2['state_holiday'].unique()

In [ ]:
# state holiday
# a = public holiday, b = Easter holiday, c = Christmas, 0 = None
df2['state_holiday']=df2['state_holiday'].apply(lambda x: 'public holiday' if x=='a' else 'Easter holiday' if x=='b'  else 'Christmas' if x=='c' else 'regular_day')
df2['state_holiday'].unique()

# 3.0. PASSO 03 - FILTRAGEM DE VARIÁVEIS

In [ ]:
df3=df2.copy()

In [ ]:
df3['assortment'].unique()

In [ ]:
df3.sample()

## 3.1 Filtragem das linhas

In [ ]:
df3 = df3[( df3['open'] != 0) & (df3['sales']>0)]

## 3.2 Seleção das colunas

In [ ]:
cols_drop = ['customers', 'open', 'promo_interval', 'month_map']
df3 = df3.drop(cols_drop, axis=1) #axis=1 são colunas

In [ ]:
df3.columns

# 4.0. PASSO 04 - ANÁLISE EXPLORATÓRIA DE DADOS

### 4.1. Análise Univariada

In [ ]:
df4 = df3.copy()

In [ ]:
df4['assortment'].unique()

#### 4.1.1. Response Variable

In [ ]:
sns.displot(df4['sales'])

In [ ]:
sns.displot(df4['sales'])

In [ ]:
sns.distplot(np.log(df4['sales']))

#### 4.1.2. Numerical Variable

In [ ]:
num_attributes.hist(bins=25)

In [ ]:
cat_attributes.sample(5)

In [ ]:
df4['state_holiday'].drop_duplicates()

In [ ]:
#df4[df4['state_holiday']!='public holiday']

In [ ]:
#sns.countplot(df4['state_holiday'], orient='v')

In [ ]:
#state_holiday,store_type,assortment

#### 4.1.3. Categorical Variable

In [ ]:
df4['state_holiday'].drop_duplicates()

In [ ]:
df4['state_holiday'].sample(4)

In [ ]:
df4[(df4['state_holiday']!='regular_day')&(df4['state_holiday']!='public holiday')&(df4['state_holiday']!='Easter holiday')].sample(5)

In [ ]:
df5=df4[df4['state_holiday'] != 'regular_day']

In [ ]:
# lista diferente de 'regular_day'
df5.state_holiday.unique()

In [ ]:
#recebendo o tamanho da lista
for i in range(len(df4.state_holiday.unique())): # identificando o tamanho da lista
    if df4.state_holiday.unique()[i]!='regular_day': # verificando o valor do indice
        print(df4.state_holiday.unique()[i]) #visualizando o valor
        print (i) # mostrando o indice

In [ ]:
plots(df4['state_holiday'], 2)

In [ ]:
#no primeiro plot não visualizo a categoria Christmas no grafico
#no segundo plot não visualiza a legenda das categorias

In [ ]:
#df4[df4['state_holiday']==label]['sales']

In [ ]:
#public_holiday

In [ ]:
df5.shape

In [ ]:
# to remove
#plot(df4['state_holiday'],
#def plot(df,
#a = df4[df4['state_holiday']!='regular_day'] #'regular_day' , KeyError: "None of [Index(['regular_day'
###a = df4[df4['state_holiday']!='regular_day'] #'regular_day' , KeyError: "None of [Index(['regular_day'
###plt.subplot(1,2,1)
###sns.countplot(a.state_holiday, orient='h') #orient=None , “v” | “h” | “x” | “y”
###a.state_holiday.unique()
# não apresenta informação de 'Christmas'] no grafico

###plt.subplot(1,2,2)
###for i in range(len(df4.state_holiday.unique())):
    #grafico com as
    ###label=df4.state_holiday.unique()[i]

    ###if label != 'regular_day':
        #print(df4.state_holiday.unique()[i])
        ###sns.kdeplot(df4[df4['state_holiday']==label]['sales'], label=label, fill=True)
        #print(i)
# o grafico 1,
# o grafico 2, não visualiza a legenda

In [ ]:
# to remove
##plt.subplot(1,2,2)
##label='public holiday'
#sns.kdeplot(df4[df4['state_holiday']==label]['sales'], label=label, shade=True) #'Easter holiday, Christmas
# `shade` is now deprecated in favor of `fill`; setting `fill=True`.
#This will become an error in seaborn v0.14.0; please update your code.
##sns.kdeplot(df4[df4['state_holiday']==label]['sales'], label=label, fill=True) #'Easter holiday, Christmas
#o grafico sobreposto e transparente facilita a observação do pico e largura na distribuição das variáveis
# visualizar as distribuições sobre postas


In [ ]:
# to remove
#plot(df4['state_holiday'],
#def plot(df,
#a = df4[df4['state_holiday']!='regular_day'] #'regular_day' , KeyError: "None of [Index(['regular_day'
###a = df4[df4['state_holiday']!='regular_day'] #'regular_day' , KeyError: "None of [Index(['regular_day'
###plt.subplot(1,2,1)
###sns.countplot(a.state_holiday, orient='h') #orient=None , “v” | “h” | “x” | “y”
###a.state_holiday.unique()
# não apresenta informação de 'Christmas'] no grafico

###plt.subplot(1,2,2)
###for i in range(len(df4.state_holiday.unique())):
    #grafico com as
    ###label=df4.state_holiday.unique()[i]

    ###if label != 'regular_day':
        #print(df4.state_holiday.unique()[i])
       ### sns.kdeplot(df4[df4['state_holiday']==label]['sales'], label=str(label), fill=True)
        #print(i)
# o grafico 1,
# o grafico 2, não visualiza a legenda

In [ ]:
#plot(df4['state_holiday'],
#def plot(df,
#a = df4[df4['state_holiday']!='regular_day'] #'regular_day' , KeyError: "None of [Index(['regular_day'
a = df4[df4['state_holiday']!='regular_day'] #'regular_day' , KeyError: "None of [Index(['regular_day'
plt.subplot(1,2,1)
sns.countplot(a,x='state_holiday', orient='h') #orient=None , “v” | “h” | “x” | “y”
a.state_holiday.unique()
# não apresenta informação de 'Christmas'] no grafico

plt.subplot(1,2,2)
for i in range(len(df4.state_holiday.unique())):
    #grafico com as
    label=df4.state_holiday.unique()[i]

    if label != 'regular_day':
        #print(df4.state_holiday.unique()[i])
        g=sns.kdeplot(df4[df4['state_holiday']==label]['sales'], label=str(label), fill=True)
        #print(i)
g.legend(loc="upper right")

# o grafico 1,
# o grafico 2, não visualiza a legenda

In [ ]:
# to remove
#plot(df4['store_type'],
#def plot(df,
#a = df4[df4['store_type']!='regular_day'] #'regular_day' , KeyError: "None of [Index(['regular_day'
#a = df4[df4['store_type']!='regular_day'] #'regular_day' , KeyError: "None of [Index(['regular_day'
###plt.subplot(1,2,1)
###sns.countplot(df4,x='store_type', orient='h') #orient=None , “v” | “h” | “x” | “y”
###a.store_type.unique()
# não apresenta informação de 'Christmas'] no grafico

###plt.subplot(1,2,2)
###for i in range(len(df4.store_type.unique())):
    #grafico com as
   ### label=df4.state_holiday.unique()[i]

    #if label != 'regular_day':
        #print(df4.state_holiday.unique()[i])
    ###g=sns.kdeplot(df4[df4['store_type']==label]['sales'], label=str(label), fill=True)
        #print(i)
###g.legend(loc="upper right")

# o grafico 1,
# o grafico 2, não visualiza a legenda

In [ ]:
#plot(df4['store_type'],
#def plot(df,
#a = df4[df4['store_type']!='regular_day'] #'regular_day' , KeyError: "None of [Index(['regular_day'
#a = df4[df4['assortment']!='regular_day'] #'regular_day' , KeyError: "None of [Index(['regular_day'
plt.subplot(1,2,1)
sns.countplot(df4,x='store_type', orient='h') #orient=None , “v” | “h” | “x” | “y”
a.store_type.unique()
# não apresenta informação de 'Christmas'] no grafico

plt.subplot(1,2,2)
for i in range(len(df4.store_type.unique())):
    #grafico com as
    label=df4.store_type.unique()[i]

    #if label != 'regular_day':
        #print(df4.store_type.unique()[i])
    g=sns.kdeplot(df4[df4['store_type']==label]['sales'], label=str(label), fill=True)
        #print(i)
g.legend(loc="upper right")

# o grafico 1,
# o grafico 2, não visualiza a legenda

In [ ]:
#plot(df4['assortment'],
#def plot(df,
#a = df4[df4['assortment']!='regular_day'] #'regular_day' , KeyError: "None of [Index(['regular_day'
#a = df4[df4['assortment']!='regular_day'] #'regular_day' , KeyError: "None of [Index(['regular_day'
plt.subplot(1,2,1)
sns.countplot(df4,x='assortment', orient='h') #orient=None , “v” | “h” | “x” | “y”
a.assortment.unique()
# não apresenta informação de 'Christmas'] no grafico

plt.subplot(1,2,2)
for i in range(len(df4.assortment.unique())):
    #grafico com as
    label=df4.assortment.unique()[i]

    #if label != 'regular_day':
        #print(df4.assortment.unique()[i])
    g=sns.kdeplot(df4[df4['assortment']==label]['sales'], label=str(label), fill=True)
        #print(i)
g.legend(loc="upper right")

# o grafico 1,
# o grafico 2, não visualiza a legenda

In [ ]:
df4.dtypes

In [ ]:
# to remove
#store_type
#change state_holiday to store_type object
#a = df4[df4['store_type']] #'regular_day' , KeyError: "None of [Index(['regular_day'
#plt.subplot(1,2,1)# change 1 to second line
#plt.subplot(2,2,1)

In [ ]:
# to remove
#change state_holiday to store_type object
###plt.subplot(1,2,1)
###sns.countplot(df4["store_type"], orient='h') #orient=None , “v” | “h” | “x” | “y”
#a.state_holiday.unique()
# não apresenta informação de 'Christmas' no grafico

##plt.subplot(1,2,2) #change 1 to 2
#plt.subplot(2,2,2)

###plt.subplot(1,2,2)

###try_="df4."  + "store_type" + ".unique()"
#for i in range(len(df4.state_holiday.unique())): change state_holiday to store_type#
#for i in range(len(df4.store_type.unique())):
###for i in range(len(try_)):
    #grafico com as
    #label=df4.state_holiday.unique()[i] change state_holiday to store_type###
    ###label=df4.store_type.unique()[i]

    ###if label != 'regular_day':
        #print(df4.state_holiday.unique()[i])
        #sns.kdeplot(df4[df4['state_holiday']==label]['sales'], label=label, fill=True) change state_holiday to store_type
        ###sns.kdeplot(df4[df4['store_type']==label]['sales'], label=label, fill=True)
    ###sns.kdeplot(df4[df4['store_type']==label]['sales'], label=label, fill=True)
        #print(i)
# o grafico 1,
# o grafico 2, não visualiza a legenda
#assortment                              object
#df4.store_type.unique()

In [ ]:
# to remove
#store_type
#change state_holiday to store_type object
####a = df4[df4['store_type']!='regular_day'] #'regular_day' , KeyError: "None of [Index(['regular_day'
#plt.subplot(1,2,1) change 1 to second line
####plt.subplot(2,2,1)

#change state_holiday to store_type object
####sns.countplot(a.store_type, orient='h') #orient=None , “v” | “h” | “x” | “y”
#a.state_holiday.unique()
# não apresenta informação de 'Christmas'] no grafico

#plt.subplot(1,2,2) change 1 to 2
####plt.subplot(2,2,2)
#for i in range(len(df4.state_holiday.unique())): change state_holiday to store_type
####for i in range(len(df4.store_type.unique())):
    #grafico com as
    #label=df4.state_holiday.unique()[i] change state_holiday to store_type
    ####label=df4.store_type.unique()[i]

    ####if label != 'regular_day':
        #print(df4.state_holiday.unique()[i])
        #sns.kdeplot(df4[df4['state_holiday']==label]['sales'], label=label, fill=True) change state_holiday to store_type
        ####sns.kdeplot(df4[df4['store_type']==label]['sales'], label=label, fill=True)
        #print(i)
# o grafico 1,
# o grafico 2, não visualiza a legenda
#assortment                              object
#df4.store_type.unique()

### 4.1.3. Categorical Variable, diamonds = sns.load_dataset('diamonds')

In [ ]:
#https://www.youtube.com/watch?v=8U5h3EJuu8M
#Seaborn countplot | What is the countplot? | Seaborn countplot vs barplot
diamonds = sns.load_dataset('diamonds')
diamonds.sample()

In [ ]:
diamonds.clarity.isin(['SI1', 'VS2']).shape

In [ ]:
diamonds[diamonds.clarity.isin(['SI1', 'VS2'])].shape

In [ ]:
sns.set_style('darkgrid')
plt.subplot(1,2,1)
sns.countplot(x='color', data=diamonds)
plt.subplot(1,2,2)
sns.countplot(x='color', data=diamonds[diamonds.clarity.isin(['SI1', 'VS2'])])

In [ ]:
diamonds.dtypes

In [ ]:
plt.subplot(1,3,1)
#cut
sns.countplot(diamonds.cut) #, orient="y") #orient“v” | “h” | “x” | “y”
#color
plt.subplot(1,3,2)
sns.countplot(diamonds.color) #, orient="y")
#diamonds.color.drop_duplicates()
diamonds['color'].drop_duplicates()
#clarity

In [ ]:
diamonds.color.cat.categories

In [ ]:
diamonds.cut.cat.categories

In [ ]:
diamonds.clarity.cat.categories

### 4.2. Análise Bivariada

### **2.2. Lista Final de Hipóteses**

#### **H1. Lojas com maior sortimento deveriam vender mais.**
#### **FALSA**, lojas com MAIOR SORTIMENTO vendem MENOS
[1., 3.] Lojas com [maior sortimento/ competidores à + tempo] deveriam vender +.

In [ ]:
#[1., 3.] Lojas com [maior sortimento/ competidores à + tempo] deveriam vender +.
aux1 = df4[['assortment', 'sales']].groupby('assortment').sum().reset_index()
#soma de todas as vendas que tem o assortment do tipo ...
sns.barplot(x='assortment', y='sales', data=aux1);

#### assortment do tipo basic e o extended tem mais ou menos o mesmo tipo de vendas e o extra tem o volume de vendas bem menor
supondo que o extra é o maior assortment vemos um tendencia decrescente
Será que houve uma mudança do comportamento ao longo do tempo

In [ ]:
# como o tempo, semanas do ano, afecta o fenomeno do assortment relativamente as vendas
aux2 = df4[['year_week', 'assortment', 'sales']].groupby(['year_week','assortment']).sum().reset_index()

In [ ]:
aux2.pivot(index='year_week', columns='assortment', values='sales').plot()

In [ ]:
#o assortment extra sugere uma linha linear e horizontal

In [ ]:
aux3=aux2[aux2['assortment']=='extra']

In [ ]:

aux3.pivot(index='year_week', columns='assortment', values='sales').plot()

2.2. Lista Final de Hipóteses
[0, 3.] Lojas com [0 / competidores à + tempo] deveriam vender +.

[2., ] Lojas com competidores + próximo deveriam vender - .

2.2. Lista Final de Hipóteses
[1., 3.] Lojas com [maior sortimento/ competidores à + tempo] deveriam vender +.





** Produtos **

[4., 5., 6.] Lojas [promoções activas por mais tempo, com + [dias de promoção, promoções consecutivas]] deveriam vender +

** Tempo **

7. Lojas abertas durante o feriado de Natal deveriam vender menos.

8. Lojas ao longo do ano deveriam vender mais.

9. Lojas no segundo semestre do ano deveriam vender mais.

10. Lojas depois do dia 10 de cada mes deveriam vender mais.

11. Lojas nos finais de semana deveriam vender menos.

12. Lojas nos Feriados escolares deveriam vender menos.

#### **H2. Lojas com competidores + próximos deveriam vender menos.**
#### **FALSA, Lojas com competidores + próximos deveriam vender mais.**


In [ ]:
#### **2.2. Lista Final de Hipóteses**
#### **H1. Lojas com maior sortimento deveriam vender mais.**
#### **FALSA, lojas com MAIOR SORTIMENTO vendem MENOS**

In [ ]:
df4.sample()

In [ ]:
com_dis='competition_distance'
y = 'sales'

bins = list(np.arange(0, 20000, 1000))
com_dis_bin='competition_distance_binned'

aux1[com_dis_bin]=pd.cut(df4[com_dis], bins=bins)
aux2 = aux1[[com_dis_bin, y]].groupby(com_dis_bin).sum().reset_index()

aux3=df4[[com_dis,y]].groupby(com_dis).sum().reset_index()
plt.subplot(1,4,1)
sns.barplot(x=com_dis_bin, y=y, data=aux2); #como são as vendas por cada distância
plt.xticks(rotation=90)

plt.subplot(1,4,2)
sns.scatterplot(x=com_dis, y=y, data=aux3);
plt.xticks(rotation=90)

plt.subplot(1,4,3)
x = sns.heatmap(aux3.corr(method='pearson'), annot=True);
#bootom, top = x.get #AttributeError: 'Axes' object has no attribute 'get'
# medir a força de correlação

#### **H3. Lojas com competidores à + tempo deveriam vender +.**
#competition_open_since, competition_time_month, competition_open_since_month, competition_open_since_year
#### **False, lojas com COMPETIDORES à + tempo vendem MENOS

In [ ]:
df4.sample()

In [ ]:
#competition_open_since_month
com_ope_sin_mon='competition_open_since_month'
y = 'sales'

#bins = list(np.arange(0, 20000, 1000))
#com_dis_bin='competition_distance_binned'
#aux1[com_dis_bin]=pd.cut(aux1[com_dis], bins=bins)
#aux2 = aux1[[com_dis_bin, y]].groupby(com_dis_bin).sum().reset_index()

aux3=df4[[com_ope_sin_mon,y]].groupby(com_ope_sin_mon).sum().reset_index()
plt.subplot(1,4,1)
sns.barplot(x=com_ope_sin_mon, y=y, data=aux3);
#plt.xticks(rotation=90)

In [ ]:
#competition_open_since_month
comp_tim_mon='competition_time_month'
y = 'sales'

#bins = list(np.arange(0, 20000, 1000))
#com_dis_bin='competition_distance_binned'
#aux1[com_dis_bin]=pd.cut(aux1[com_dis], bins=bins)
#aux2 = aux1[[com_dis_bin, y]].groupby(com_dis_bin).sum().reset_index()

aux1=df4[['competition_time_month',y]].groupby('competition_time_month').sum().reset_index()
plt.subplot(1,4,1)


#filtrando, todas as linhas e colunas, os primeiros 12 meses de competição e com tempo de competição diferente ao da realização da venda
aux2 = aux1[( aux1['competition_time_month']<12) & (aux1[comp_tim_mon]!=0)]
sns.barplot(x=comp_tim_mon, y=y, data=aux2);
#plt.xticks(rotation=60)

plt.subplot(1,4,2)
#exite alguma tendencia de crescimento?
sns.regplot(x=comp_tim_mon, y=y, data=aux2);

plt.subplot(1,4,3)
#sns.heatmap(x='competition_time_month', y=y, data=aux2);
#em termos de relevância para o modelo,
# é relevante pois tem uma correlação não tão próxima de zero
x = sns.heatmap(aux1.corr(method='pearson'), annot=True);



In [ ]:
#Quanto + recente forem as competições, a abertura das lojas do competidores, maior são as suas vendas.

#### **H4. Lojas com promoções activas por mais tempo, com + dias de promoção, deveriam vender + **
#### Falsa, lojas com promoções activas por mais tempo vendem menos, depois de um certo período de promoção
#### vende regularmente até um certo tempo, depois decai
#### validar a hipotese vs
#### a variável é relevante para o modelo ?

In [ ]:
#### **H4. Lojas promoções activas por mais tempo, com + dias de promoção deveriam vender + o **
df4[['date','promo','is_promo','promo2', 'promo2_since_week','promo2_since_year', 'promo_since', 'promo_time_week']].sample(15)
#df4.sample()#.co
#['promo2', 'promo2_since_week',	'promo2_since_year', 'promo_since', 'promo_time_week']
aux1=df4[['promo_time_week','sales']].groupby('promo_time_week').sum().reset_index()
sns.barplot(x='promo_time_week', y='sales', data=aux1);
#promo_since	promo_time_week
#promo2	promo2_since_week	promo2_since_year

In [ ]:
y='promo_time_week'
x='sales'
aux2=aux1[aux1['promo_time_week']>0] # Promo extendido
aux3=aux1[aux1['promo_time_week']<0] # Promo regular, tradicional

#plt.subplot(1,2,1)
#sns.barplot(x='promo_time_week', y='sales', data=aux2, label='Promo extendido ');
#plt.xticks(rotation=90);

plt.subplot(1,2,1)
sns.barplot(x='promo_time_week', y='sales', data=aux3, label='Promo regular, tradicional');
#plt.subplot(1,2,1)
plt.xticks(rotation=90);

plt.subplot(1,2,2)
sns.barplot(x='promo_time_week', y='sales', data=aux2, label='Promo extendido ');
plt.xticks(rotation=90);


In [ ]:
plt.subplot(2,1,2)
sns.barplot(x='promo_time_week', y='sales', data=aux2, label='Promo extendido ');
plt.xticks(rotation=90);

plt.subplot(2,1,1)
sns.barplot(x='promo_time_week', y='sales', data=aux3, label='Promo regular, tradicional ');
#plt.subplot(1,2,1)
plt.xticks(rotation=90);

#plt.subplot(1,2,2)
#sns.barplot(x='promo_time_week', y='sales', data=aux2, label='Promo extendido ');
#plt.xticks(rotation=90);


In [ ]:
# os gráficos acima sugerem uma tendência
#verificando se realmente existe alguma tendência nos gráficos
plt.subplot(1,2,1)
sns.regplot(x='promo_time_week', y='sales', data=aux3, label='Promo regular, tradicional');
#plt.subplot(1,2,1)
plt.xticks(rotation=90);

plt.subplot(1,2,2)
sns.regplot(x='promo_time_week', y='sales', data=aux2, label='Promo extendido ');
plt.xticks(rotation=90);


In [ ]:
# Procurando encontrar a força do comportamento da correlação, usado o método corr, obedecendo a formula de pearson 19:31,
sns.heatmap(aux1.corr( method='pearson'), annot=True);
# correlação de 0.029 é uma correlação muito fraca

In [ ]:
# Procurando encontrar a força do comportamento da correlação, usado o método corr, obedecendo a formula de pearson 19:31,
sns.heatmap(aux2.corr( method='pearson'), annot=True);
# correlação de 0.029 é uma correlação negativa e forte

In [ ]:
# Procurando encontrar a força da correlação, usado o método corr, obedecendo a formula de pearson 19:31,
sns.heatmap(aux3.corr( method='pearson'), annot=True);
# correlação de 0.94 é uma correlação positiva e muito forte

In [ ]:
# grid=gridspec.GridSpec(2,3) #2 linhas e 3 colunas
grid=GridSpec(2,3) #2 linhas e 3 colunas

plt.subplot(grid[0,0])
# aux2=

x='promo_time_week'
y='sales'

# ValueError: Could not interpret value `promo_time_week` for `x`. An entry with this name does not appear in `data`.
# aux1.sample()
sns.barplot(x=x, y=y, data=df4);
#plt.xticks(rotation=90)

plt.subplot(grid[0,1])
sns.regplot(x=x, y=y, data=aux2);

plt.subplot(grid[1,0])
sns.barplot(x=x, y=y, data=aux3);

plt.subplot(grid[1,1])
sns.regplot(x=x, y=y, data=aux3);

plt.subplot(grid[:,2])
sns.heatmap(aux1.corr(method='pearson'), annot=True)

In [ ]:
#aux3

#### ** <s> H5. Lojas com + dias de promoção, deveriam vender + #### ** Tempo **  </s>
#### ** H4. Lojas com promoções activas por mais tempo, com + dias de promoção, deveriam vender + **
#### Falsa, lojas com promoções activas por mais tempo vendem menos, depois de um certo período de promoção
#### vende regularmente até um certo tempo, depois decai
#### validar a hipotese vs

In [ ]:
df4.sample(5)

#### **H6. Lojas com + dias de promoções consecutivas deveriam vender +
#### ** FALSA ** , Lojas com + promoções consecutivas vendem menos

In [ ]:
df4[['promo','promo2','sales']].groupby(['promo','promo2']).sum().reset_index().sort_values(by=['sales'], ascending=True) #df.sort_values(by=['Column_name'], ascending=True)

In [ ]:
# a diferença existente entre as 4 vendas é significativa ?

In [ ]:
#será que ao longo do tempo, o comportamento das vendas mudou tendo activado as duas promoções, a regular e a tradicional?
df4[((df4['promo']==1) & (df4['promo2']==1))]

In [ ]:
aux1=df4[( df4['promo'] == 1 ) & ( df4['promo2'] == 1 )][['year_week', 'sales']]
aux1.plot()

In [ ]:
#agrupar, somar e fazer o reset_index
aux2=df4[( df4['promo'] == 1 ) & ( df4['promo2'] == 1 )][['year_week', 'sales']].groupby('year_week').sum().reset_index()
aux2.plot()

In [ ]:
aux3=df4[( df4['promo'] == 1 ) & ( df4['promo2'] == 0 )][['year_week', 'sales']].groupby('year_week').sum().reset_index()
#ax = aux3.plot()
aux4=df4[( df4['promo'] == 0 ) & ( df4['promo2'] == 0 )][['year_week', 'sales']].groupby('year_week').sum().reset_index()
aux5=df4[( df4['promo'] == 0 ) & ( df4['promo2'] == 1 )][['year_week', 'sales']].groupby('year_week').sum().reset_index()
ax = aux3.plot()
aux2.plot(ax=ax)
aux4.plot(ax=ax)
aux5.plot(ax=ax)
ax.legend(labels=("Tradicional & Extraordinario","Tradicional","Sem promoção","Extraordinario"))

#### ** H7. Lojas abertas durante o feriado de natal deveriam vender menos. **
#### ** ** Verdade, Lojas abertas durante o feriado de natal vendem menos,
#### quanto a relevância,
#### mostra-se relevante dado o crescimento considerável de vendas para cada 1 dos feriados

In [ ]:
march_dates = (
        df4['date'].dt.month.eq(12) &
        df4['date'].dt.day.between(25, 31)
)
april_dates = (
        df4['date'].dt.month.eq(4) &
        df4['date'].dt.day.between(1, 25)
)
m = march_dates #| april_dates

filtered_df = df4[m][['date','sales']]#.groupby('date').sum().reset_index()
filtered_df.plot(y='sales',x='date')

In [ ]:
march_dates = (
        df4['date'].dt.month.eq(12) &
        df4['date'].dt.day.between(25, 31)
)
april_dates = (
        df4['date'].dt.month.eq(4) &
        df4['date'].dt.day.between(1, 25)
)
m = march_dates #| april_dates

filtered_df = df4[m][['date','sales']]#.groupby('date').sum().reset_index()
filtered_df.plot(y='sales',x='date')


In [ ]:
#df4[df4['state_holiday']!='regular_day'][['date','sales','state_holiday','school_holiday']].groupby('state_holiday').sum().reset_index()#.sample(5)
#aux=df4[df4['state_holiday']!='regular_day'][['sales','state_holiday']].groupby('state_holiday').sum().reset_index()#.sample(5)
#aux.plot(y='sales',x='state_holiday')
#sns.catplot(kind="bar", x='state_holiday',y='sales', data=aux)
#
#cols=pd.unique(df4['state_holiday'])
aux2=df4[df4['state_holiday']!='regular_day'][['sales','state_holiday','year']].groupby('year').sum().reset_index()#.sample(5)
#g =
#sns.catplot(kind="bar", x='year',y='sales', data=aux2, hue='state_holiday', alpha=.4, height=4) #label=cols )#'state_holiday')

#RuntimeError: In draw_glyphs_to_bitmap: Could not convert glyph to bitmap (raster overflow; error code 0x62

#g.fig.set_figheight(3.5)
#g.fig.set_figwidth(3)AU

#g.despine(left=True)

In [ ]:
aux3=df4[df4['state_holiday']!='regular_day'][['sales','state_holiday','year']].groupby(['state_holiday', 'year']).sum().reset_index()#.sample(5)

In [ ]:
aux3

In [ ]:
aux2

In [ ]:
#ValueError: Could not interpret value `year` for `x`. An entry with this name does not appear in `data`.
aux2.sample()

In [ ]:
plt.subplot(2,1,1)
#hue=
#sns.catplot(kind="bar", x='year',y='sales', data=aux2) #, alpha=.4, height=4) #label=cols )#'state_holiday')
#sns.catplot(kind="bar", x='year',y='sales', data=aux2) #, alpha=.4, height=4) #label=cols )#'state_holiday')
#sns.barplot(x='year', y='sales', hue='state_holiday', data=aux2, alpha=0.7)

#ValueError: Could not interpret value `year` for `x`. An entry with this name does not appear in `data`.
#year_week	sales
#sns.barplot(x='year', y='sales', data=aux2, alpha=0.7)

#ValueError: Could not interpret value `year_week` for `x`. An entry with this name does not appear in `data`.
sns.barplot(x='year_week', y='sales', data=aux2, alpha=0.7)
#ValueError: Could not interpret value `year` for `hue`. An entry with this name does not appear in `data`.
plt.show()
plt.subplot(2,1,2)
#hue=
#sns.catplot(kind="bar", x='state_holiday',y='sales', data=aux3) #, alpha=.4, height=4) #label=cols )#'state_holiday')
sns.barplot(x='state_holiday', y='sales', hue='year', data=aux3, alpha=0.7)

#?? todo: to see sns.catplot vs sns.barplot
#barplot
#Show point estimates and confidence intervals using bars.
#catplot
#Combine a categorical plot with a FacetGrid.

#?? todo: aux4:count, quantos feriados públicos temos/ano
plt.show()


#### ** H8. Lojas ao longo do ano deveriam vender +
#### ** ** Falsa, as lojas ao longo do ano vendem -
#### Quanto a relevância,
#### a variável ano é ou não relevante para o modelo
#### existe uma variação acentuada das vendas ao longo dos anos?
#### o ano de 2015 tem vendas de meses em falta
#### o ano tem 12 meses contudo o 2015 tem vendas em apenas x meses
#### que tal montar uma serie que identifica o data e consider as vendas e fazer uma previsão para os meses #### em falta, fazer as análises considerando os períodos idénticos


In [ ]:
#as vendas do ano 2015 terminam em Agosto, pode ser a causa de baixa venda comparando com os restantes anos

In [ ]:
#KeyError: 'year'
#aux1=promo&promo2
aux1=df4[( df4['promo'] == 1 ) & ( df4['promo2'] == 1 )][['year', 'sales']]
aux1.sample()

In [ ]:
#Falta o primeiro grafico

In [ ]:
aux2.sample()

In [ ]:
#KeyError: 'promo_time_week'
aux2.sample()
# de
# year_week	sales
# para
# year	sales	state_holiday

In [ ]:
# grid=gridspec.GridSpec(2,3) #2 linhas e 3 colunas
grid=GridSpec(2,3) #2 linhas e 3 colunas

plt.subplot(grid[0,0])
# aux2=

x='promo_time_week'
y='sales'

# ValueError: Could not interpret value `promo_time_week` for `x`. An entry with this name does not appear in `data`.
# aux1.sample()
##KeyError: 'year'
sns.barplot(x=x, y=y, data=df4);
#plt.xticks(rotation=90)

#plt.subplot(1,3,1)
#aux2=df4[df4['state_holiday']!='regular_day'][['sales','state_holiday','year']].groupby('year').sum().reset_index()#.sample(5)
#g =
#sns.catplot(kind="bar", x='year',y='sales', data=df4) #aux2, )#hue='state_holiday', alpha=.4, height=4) #label=cols )#'state_holiday')
#https://seaborn.pydata.org/examples/grouped_barplot.html

plt.subplot(grid[0,1])
##KeyError: 'promo_time_week'
x='year_week'
#year_week	sales
## The above exception was the direct cause of the following exception:
## UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching
## types (dtype('<U7'), dtype('float64')) -> None
##trocar o tipo de grafico
###sns.regplot(x=x, y=y, data=aux2);
#plt.subplot(1,3,2)

#KeyError: 'year'
#sns.regplot(x='year', y='sales', data=aux1, label='Promo extendido ');


#plt.xticks(rotation=90);


#plt.subplot(grid[1,0])
#sns.barplot(x=x, y=y, data=aux3);
#ValueError: Could not interpret value `year_week` for `x`. An entry with this name does not appear in `data`.
sns.barplot(x=x, y=y, data=aux2);

plt.subplot(grid[0,2])
#sns.regplot(x=x, y=y, data=aux3);

#plt.subplot(grid[:,2])
#sns.heatmap(aux1.corr(method='pearson'), annot=True)
#plt.subplot(1,3,3)

aux1=df4[['sales','year']].groupby('year').sum().reset_index()
# aux1 todas as vendas independente do período
# verificando o grau de correlação entre as variáveis year and sales
sns.heatmap(aux1.corr(method='pearson'), annot=True);

#g.fig.set_figheight(3.5)
#g.fig.set_figwidth(3)

#g.despine(left=True)



#### ** H9. Lojas no 2o semestre deveriam vender + #### ** **
#### Falsa, as lojas no 2o semestre vendem -,
#### cada 1 dos meses tem vendas inferiores a 5.0 no 1o semestre do acumulado de vendas por mês é superior a 5.0 logo
#### as vendas do 1 semestre são superiores as do 2o semestre
#### contudo, como o ano 2015 esta imcompleto, sugere-se a adicinas os meses de 1 -> 7,
#### isto é fazer-se uma análise exploratória apenas com os dados dos anos 2013 e 2014, pois estão completos

In [ ]:
#procurando valor do ano
#df4['year'].sample()
#não era necessário considerar os meses no intervalo de 1 a 12
#aux=df4[df4['month'].between(1, 12)][['month','sales','year']].groupby(['year','month']).sum().reset_index()
aux=df4[df4['month'].between(1, 12)][['month','sales','year']].groupby(['year','month']).sum().reset_index()
#criar dois tipos de dados, anos completos e incompletos de  2015
plt.subplot(4,1,1)
sns.pointplot(x='year',y='sales', hue='month', data=aux)
plt.subplot(4,1,2)
sns.pointplot(x='year',y='sales',data=aux)

aux1=df4[['month','sales','year']].groupby(['year','month']).sum().reset_index()
plt.subplot(4,1,3)
sns.pointplot(x='year',y='sales', hue='month', data=aux1)
plt.subplot(4,1,4)
sns.pointplot(x='year',y='sales',data=aux1)

In [ ]:
#como ter informação do segundo semestre
#df4.sample()
#agrupar por mes
plt.subplot(4,1,1)
aux=df4[df4['month'].between(1, 12)][['month','sales','year']].groupby(['month','year']).sum().reset_index()
#df4[df4['month'].between(1, 12)][['month','sales','year']].groupby('month').sum().reset_index()
sns.pointplot(x='month',y='sales',data=aux)

#considerar as vendas por ano
plt.subplot(4,1,2)
#agrupar por ano
sns.pointplot(x='year',y='sales', hue='month',data=aux)
semestre1=df4[df4['date'].dt.month.between(1, 6)][['month','sales','year']].groupby(['month','year']).sum().reset_index()
semestre2=df4[df4['date'].dt.month.between(7, 12)][['month','sales','year']].groupby(['month','year']).sum().reset_index()
#df4['date'].dt.month.between(1, 6)
plt.subplot(4,1,3)
sns.pointplot(x='month',y='sales',data=semestre1, label='primeiro semestre')

plt.subplot(4,1,4)
sns.pointplot(x='month',y='sales',data=semestre2, label='segundo semestre')

In [ ]:
#como ter informação do segundo semestre
#df4.sample()
#agrupar por mes
plt.subplot(4,1,1)
aux=df4[df4['month'].between(1, 12)][['month','sales','year']].groupby(['month','year']).sum().reset_index()
#df4[df4['month'].between(1, 12)][['month','sales','year']].groupby('month').sum().reset_index()
sns.pointplot(x='month',y='sales',data=aux)

#considerar as vendas por ano
plt.subplot(4,1,2)
#agrupar por ano
sns.pointplot(x='year',y='sales', hue='month',data=aux)
semestre1=df4[df4['date'].dt.month.between(1, 6)][['month','sales','year']].groupby(['month','year']).sum().reset_index()
semestre2=df4[df4['date'].dt.month.between(7, 12)][['month','sales','year']].groupby(['month','year']).sum().reset_index()
#df4['date'].dt.month.between(1, 6)
plt.subplot(4,1,3)
sns.pointplot(x='month',y='sales',data=semestre1, label='primeiro semestre')

plt.subplot(4,1,4)
sns.pointplot(x='month',y='sales',data=semestre2, label='segundo semestre')

In [ ]:
#os meses do ano 2015 sem vendas são de 8 a 12
aux.sample()

#### ** H10. Lojas do dia 10 de cada mês deveriam vender + **
#### ** **

In [ ]:
#df4.sample()
#linha vs coluna
#vendas vs day
day_sales=df4[['day','sales']].groupby('day').sum().reset_index()
day_sales.sample()
# todo: grafico de [barra ,regressão,correlação](day vs sales) e bara(before_or_after_10 vs sales)

In [ ]:
aux.sample(10)

In [ ]:
#quais os dados que temos?
df4.sample()

In [ ]:

#df4['after_or_before_10']=df4['day'].apply(lambda x: 'before' if x['day']<11 else 'after')
#between
#TypeError: 'int' object is not subscriptable
# x['date'], TypeError: 'Timestamp' object is not subscriptable
#apply
#df2['state_holiday']=df2['state_holiday'].apply(lambda x: 'public holiday' if x=='a' else 'Easter holiday' if x=='b'  else 'Christmas' if x=='c' else 'regular_day')
df4['after_or_before_10']=df4['day'].apply(lambda x: 'before' if x<=10 else 'after')
df4['after_or_before_10']

In [ ]:
#como ter informação do segundo semestre
#df4.sample()
#agrupar por mes
plt.subplot(4,1,1)
#aux=df4[df4['day'].between(1, 12)][['day','month','sales','year']].groupby(['day','month','year']).sum().reset_index()
aux=df4[df4['day'].between(1, 12)][['day','month','sales']].groupby(['day','month']).sum().reset_index()
#df4[df4['month'].between(1, 12)][['month','sales','year']].groupby('month').sum().reset_index()
#ValueError: Length of list vectors must match length of `data` when both are used,
# but `data` has length 31 and the vector passed to `hue` has length 2
sns.pointplot(x='day',y='sales',data=aux) #, hue=['month'])
# ValueError: Length of list vectors must match length of `data` when both are used,
# but `data` has length 31 and the vector passed to `hue` has length 1.

#considerar as vendas por ano
plt.subplot(4,1,2)
#sns.pointplot(x='month',y='sales',data=aux, hue=['day'])
#ValueError: Length of list vectors must match length of `data` when both are used,
#but `data` has length 144 and the vector passed to `hue` has length 1.
#agrupar por ano
#sns.pointplot(x='year',y='sales', hue='month',data=aux)
#apply

#TypeError: 'int' object is not subscriptable
df4['after_or_before_10']=df4['day'].apply(lambda x: 'before' if x['day']<11 else 'after')
sales_until_10=df4[df4['date'].dt.day.between(1, 10)][['day','month','sales','year']].groupby(['day','month','year']).sum().reset_index()
sales_after_10=df4[df4['date'].dt.day.between(11, 31)][['day','month','sales','year']].groupby(['day','month','year']).sum().reset_index()
#df4['date'].dt.month.between(1, 6)
plt.subplot(4,1,3)
#sns.pointplot(x='day',y='sales',data=sales_until_10, label='sales until day 10', hue=['month','year'] )
sns.pointplot(x='day',y='sales',data=sales_until_10, label='sales until day 10')#, hue=['month'] )

plt.subplot(4,1,4)
sns.pointplot(x='day',y='sales',data=sales_after_10, label='sales after day 10')#, hue=['year'] )
#sns.pointplot(x='day',y='sales',data=sales_after_10, label='sales after dia 10', hue=['month','year'])

In [ ]:
#NameError: name 'sales_after_10' is not defined
sales_after_10.sample()

In [ ]:
#sales_after_10=sales_after_10[['day','sales']].groupby(['day']).sum().reset_index()
#sales_after_10=df4[df4['date'].dt.day.between(11, 31)][['day','month','sales','year']]
#.groupby(['day','month','year']).sum().reset_index()
#sales_until_10=sales_until_10[['day','sales']].groupby(['day']).sum().reset_index() # df4[df4['date'].dt.day.between(1, 10)][['day','month','sales','year']].groupby(['day','month','year']).sum().reset_index()
#sales_after_10=df4[df4['date'].dt.day.between(11, 31)][['day','month','sales','year']].groupby(['day','month','year']).sum().reset_index()
#df4['date'].dt.month.between(1, 6)
plt.subplot(2,1,1)
#sns.pointplot(x='day',y='sales',data=sales_until_10, label='sales until day 10', hue=['month','year'] )
sns.barplot(x='day',y='sales',data=sales_until_10, label='sales until day 10')#, hue=['month'] )

plt.subplot(2,1,2)
sns.barplot(x='day',y='sales',data=sales_after_10, label='sales after day 10')#, hue=['year'] )
#sns.pointplot(x='day',y='sales',data=sales_after_10, label='sales after dia 10', hue=['month','year'])

In [ ]:
#sales_after_10=sales_after_10[['day','sales']].sum().reset_index()

#sales_until_10=sales_until_10[['day','sales']].sum().reset_index()

plt.subplot(2,1,1)
#sns.pointplot(x='day',y='sales',data=sales_until_10, label='sales until day 10', hue=['month','year'] )
sns.countplot(x='day',data=sales_until_10, label='sales until day 10')#, hue=['month'] )
#sns.countplot(x='day',y='sales',data=sales_until_10, label='sales until day 10')#, hue=['month'] )

plt.subplot(2,1,2)
#sns.countplot(x='day',y='sales',data=sales_after_10, label='sales after day 10')#, hue=['year'] )
sns.countplot(x='day',data=sales_after_10, label='sales after day 10')#, hue=['year'] )
#sns.pointplot(x='day',y='sales',data=sales_after_10, label='sales after dia 10', hue=['month','year'])

#### ** H10. Lojas aos finais de semana, deveriam vender -
#### ** **

#### ** H10. Lojas durante os feriados escolares deveriam vender -
#### ** **

#### ** H7. Lojas com + dias de promoções consecutivas deveriam vender + ** Tempo **
#### ** H7. Lojas com competidores + próximos deveriam vender menos.**

#### ** H7. Lojas com + dias de promoções consecutivas deveriam vender + ** Tempo **
#### ** H7. Lojas com competidores + próximos deveriam vender menos.**

#### **H2. Lojas com competidores + próximos deveriam vender menos.**

#### **H2. Lojas com competidores + próximos deveriam vender menos.**

#### [4., 5., 6.] Lojas [promoções activas por mais tempo, com + [dias de promoção, promoções consecutivas]] deveriam vender +
** Tempo **

### 4.3. Análise Multivariada

#### **H4. Lojas com promoções activas por mais tempo, com + dias de promoção, deveriam vender + **
#### Falsa, lojas com promoções activas por mais tempo vendem menos, depois de um certo período de promoção
# vende regularmente até um certo tempo, depois decai
# validar a hipotese vs
# a variável é relevante para o modelo ?

In [ ]:
#!git clone -b modulo04_exploratory_data_analysis --single-branch https://github.com/cabicho/ds_in_deploy.git

#fazer a importação de dum arquivo

https://colab.research.google.com/drive/13QbpDpdiNgrvYkzvcEsup2ZUxcGKYUhO#scrollTo=3q_33E5QdiEz

# 5.0. PASSO 05 - DATA PREPARATION

In [ ]:
df5=df4.copy()

##5.1 Normalization

##5.1 Rescaling
# ??? month	day day_of_week year_week-actualmente temos problemas nesta variável pois não é inteiro

###Quais as variáveis que vamos aplicar o rescaling ? Whats variables will be apply rescaling?
### Selecting numerical variables

In [ ]:

#qual a escala a ser usada, (Min-Max ou Robust) scaler
#   year	 competition_time_month  promo_time_week competition_distance
var_numerical=df5.select_dtypes(include=['int64','float64'])
var_numerical.sample(3)

In [ ]:
#df5.sample() # why year_week is not a numeric variable?
# como convertemos o promo_time_week?
#df2['promo_time_week']=((df2['date']-df2['promo_since'])/7).apply(lambda x: x.days).astype(int)

In [ ]:
lista=[ 'year','competition_time_month','promo_time_week', 'competition_distance']

In [ ]:
sns.boxplot(df5[lista])

In [ ]:
lista_much_outliers = [ 'promo_time_week', 'competition_distance']
lista_fews_outliers = ['year','competition_time_month']

In [ ]:
#encontrado os parametros da fórmula e logo aplica nos dados, obtendo assim 1 nova variável mas na mesma escala
rs  = RobustScaler()
mms = MinMaxScaler()

In [ ]:
# much outliers, muito discrepantes
def trans_robust( i):
    df5[i]=rs.fit_transform(df5[[i]].values)
    df5[i].sample(5)

for i in lista_much_outliers: #to range(len(lista)):
  trans_robust(i) #lista[i])

In [ ]:
#fews outliers, alguns valores discrepantes
def trans_minmax( i):
    df5[i]=mms.fit_transform(df5[[i]].values)
    df5[i].sample(5)
#lista_fews_outliers=['year','competition_time_month']
for i in lista_fews_outliers: #to range(len(lista)):
  trans_minmax(i) #lista[i])

In [ ]:
len(lista)

In [ ]:
#showing the distribution of variables
lista = lista_much_outliers + lista_fews_outliers
j=1
for i in lista:
  plt.subplot(len(lista)*2,1,j)
  j=j+2
  sns.histplot(df5[i])

In [ ]:
sns.boxplot(df5[lista])

## 5.1 Normalization

##5.3.1 Encoding

In [ ]:
# https://towardsdatascience.com/all-about-categorical-variable-encoding-305f3361fd02
#1.7
cat_attributes = df5.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]']) #include=['object']
obj_attributes = df5.select_dtypes(include=['object']) #include=['object']


In [ ]:
#StateHoliday - indicates a state holiday. Normally all stores, with few exceptions,
#are closed on state holidays. Note that all schools are closed on public holidays and weekends.
#a = public holiday, b = Easter holiday, c = Christmas, 0 = None
df5=pd.get_dummies(df5, prefix=['state_holiday'], columns=['state_holiday'])

#StoreType - differentiates between 4 different store models: a, b, c, d
#store_type = [a,b,c], existe uma ordem?, está em falra o re,
# que tal adicionar o feriado escolar com um dos dias esta na mesma variável que os restantes feriados
#trocando letras por números
le = LabelEncoder()
df5['store_type']=le.fit_transform(df5['store_type']) #trocando letras por números

#Assortment - describes an assortment level: a = basic, b = extra, c = extended,  exite um ordem => 3) Ordinal Encoding
assort_dict = {'basic':1, 'extra':2, 'extended':3}
#update values of assortment to ordinal encoding
df5['assortment']=df5.assortment.map(assort_dict)


In [ ]:
df5.assortment.sample(3)

##5.3.2 Response Variable Transformation

In [ ]:
sns.displot(df5.sales)

In [ ]:
#df5.sales = np.loglp.df5.sales
df5.sales = np.log1p(df5.sales)


In [ ]:
sns.displot(df5.sales) #o grafico já apresenta uma distribuição mais próxima da normal

##5.3.3 Nature Transformation

In [ ]:

#day_week, day, moth, year, week_year
#https://medium.com/@axelazara6/why-we-need-encoding-cyclical-features-79ecc3531232
new_values=[]
# to help function 5.3.3 Nature Transformation
def encode(data, col, max_val):
  sin_= col+'_sin'
  new_values.append(sin_)
  #data[col+'_sin']=np.sin(2*np.pi*data.col)/max_val
  #df5['day_of_week_sin'] = df5['day_of_week'].apply( lambda x: np.sin( x * ( 2. *np.pi/7 ) ) )
  data[sin_]=np.sin(2*np.pi*data[col])/max_val
  cos_ = col+'_cos'
  new_values.append(cos_)
  #data[col+'_cos']=np.cos(2*np.pi*data.col)/max_val
  data[cos_]=np.cos(2*np.pi*data[col])/max_val
  return data
  #print(sin_, cos_)


In [ ]:
df5.sample()

In [ ]:
# because 'year_week':52 then => can't multiply sequence by non-int of type 'float'
#dict_lista={'day_of_week':7, 'month':12, 'day':30, 'year_week':52} #,#day:31 como maximo ?, 60 week_for_year? <=max(12)meses/ano * max(5)semanas/mes #'competition_time_month'}
#df6=df5.copy() # todo coment
#listing variables to be transformed
dict_lista={'day_of_week':7, 'month':12, 'day':30} #, 'year_week':52} 356 days of year by 7 days of week then 52 weeks by year, every 52 weeks we have 1 year
for key, value in dict_lista.items():
  df5=encode(df5.copy(), key, value) #chante to df5
  #encode(df5, key, value)
  #print(key, value)

In [ ]:
#df6.sample()
#df6[new_values].sample() # chanche to df5
df5[new_values].sample() # chanche to df5

# 6.0. PASSO 06 - FEATURE SELECTION

In [ ]:
#df7=df6.copy() #
df6=df5.copy()
#df7[new_values].sample() #
df6[new_values].sample()

In [ ]:
#list(dict_lista.keys()) #+new_values
#no_in_dic_lista=['day_of_week','promo_since','competition_since','year_week'] #"['competition_since'] not in index"
no_in_dic_lista=['day_of_week','promo_since','year_week']
dict_lista_final=list(dict_lista.keys())+no_in_dic_lista
#df8=df7.copy()
#df8=df8.drop(dict_lista_final)
#df8[dict_lista_final].sample()
df6[dict_lista_final].sample()

## 6.1. Split Dataframe Into Training and Test Dataset

In [ ]:
cols_drop = dict_lista_final
df6 = df6.drop(cols_drop, axis=1)
df6.sample()

In [ ]:
day=df6[['date','store']].groupby('store').max().reset_index()['date'][0]
day #Timestamp('2013-01-01 00:00:00')
datetime.timedelta(days=6*7) #datetime.timedelta(days=42)
max_date_train = day-datetime.timedelta(days=6*7)
max_date_train#[0] # = min_date_test Timestamp('2012-11-20 00:00:00')

# training dataset
df6_train = df6[df6['date']< max_date_train] #df6['date']< '2015-06-05'] #max_date_train
#df6[df6_train].sample()
df6_train.sample(10) #608130, 969198
X_train = df6_train.copy()
y_train = X_train['sales']

# test dataset
df6_test = df6[df6['date']>= max_date_train] #df6['date']< '2015-06-05'] #max_date_train
#df6[df6_train].sample()
df6_test.sample(10) # 2015-07-17
X_test = df6_test.copy()
y_test = X_test['sales']

In [ ]:
print('Train Min Date: {}'.format(df6_train.date.min()))
print('Train Max Date: {}'.format(df6_train.date.max()))

print('\nTest Min Date: {}'.format(df6_test.date.min()))
print('Test Max Date: {}'.format(df6_test.date.max()))

## 6.1. Boruta as Feature Selection

In [ ]:
#fd6_train.sample()

#df6 = df6.drop(cols_drop, axis=1)
#df6_train.sample(10)[['date','sales']]
x_train_n = df6_train.drop(['date','sales'], axis=1).values
y_train_n = df6_train['sales'].values.ravel() # para colocar dentro de 1 vector

df6_test.drop(['date','sales'], axis=1).values #sample(10) # 2015-07-17


In [ ]:
#define train and test dataset for Boruta
#x_train_n
#y_train_n

x_train_n = df6_train.drop(['date','sales','competition_open_since','competition_distance'], axis=1).values
#df6_train.dtypes

y_train_n = df6_train['sales'].values.ravel() # para colocar dentro de 1 vector
print(" x_train_n.dtype {}, y_train_n.dtype {}".format(x_train_n.dtype,y_train_n.dtype))
df6_train.isna().sum()

In [ ]:
# define RandomForestRegressor
rf = RandomForestRegressor(n_jobs=-1)#(n_jobs=1)

In [ ]:
x_train_n #.isna().sum()

In [ ]:
y_train_n

In [ ]:
# define Boruta
# Mascarando dtypes para rodar o boruta
#np.float = float
#np.int = int
#np.object = object
#np.bool = bool
#boruta = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=42).fit(x_train_n, y_train_n)

In [ ]:
#cols_selected = boruta.support_.tolist()
#cols_selected #
#cols_selected = boruta.support_.tolist()cols_selected

cols_selected =[True,True, False, False, True, True, True, True, False, True, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False]

### 6.2.1. Best Features from Boruta

In [ ]:
#best features
#X_train_fs = X_train.drop(['date','sales'], axis=1) df6_train
X_train_fs = df6_train.drop(['date','sales','competition_open_since','competition_distance'], axis=1)
#IndexError: Boolean index has wrong length: 25 instead of 26
cols_selected_boruta = X_train_fs.iloc[:, cols_selected].columns.to_list()
cols_selected_boruta #.sample() #AttributeError: 'list' object has no attribute 'sample'
# [1:'store',2:'promo',3:'store_type',4:'assortment',5:'competition_open_since_month',6:'competition_open_since_year',7:'promo2_since_week',8:'competition_time_month',9:'promo_time_week']
# H3. Lojas com competidores à + tempo deveriam vender +.# **False, lojas com COMPETIDORES à + tempo vendem MENOS
# 4-9:'competition_open_since_month' mostrou-se relevante para o modelo, 5-8:'competition_open_since_year'
#**H6. Lojas com + dias de promoções consecutivas deveriam vender +, ** FALSA ** , Lojas com + promoções consecutivas vendem menos
# 2-9: 'promo'

In [ ]:
#not selected boruta
cols_not_selected_boruta = list(np.setdiff1d(X_train_fs.columns, cols_selected_boruta))
cols_not_selected_boruta #['day_cos','day_of_week_cos','day_of_week_sin','day_sin','id','is_promo','month_cos','month_sin','promo2',
 #'promo2_since_year','school_holiday','state_holiday_Christmas','state_holiday_Easter holiday','state_holiday_public holiday','state_holiday_regular_day','year']
#X_train_fs.columns

In [ ]:
#feature to add
feature_to_add = ['date','sales']

# final features
#cols_selected_boruta.extend(feature_to_add)

#cols_selected_boruta

# 7.0. PASSO 07 - MACHINE LEARNING MODELLING

In [ ]:
x_train = X_train[cols_selected_boruta] #X_train[cols_selected_boruta]
x_test = X_test[cols_selected_boruta]

In [ ]:
x_train.sample()

## 7.1. Average Model

In [ ]:
aux1=x_test.copy()

#add sales collumns
aux1['sales']=y_test.copy()

In [ ]:
# prediction
# calcu avarage salling of each of the stores
aux2=aux1[['store','sales']].groupby('store').mean().reset_index().rename(columns={'sales':'predictions'})

In [ ]:
aux1 = pd.merge(aux1, aux2, how='left', on='store')
yhat_baseline = aux1['predictions']

In [ ]:
#performance
baseline_result = ml_error('Average Model', np.expm1(y_test), np.expm1(yhat_baseline))

In [ ]:
baseline_result

## 7.2. Linear Regression Model

In [ ]:
# model
lr = LinearRegression().fit(x_train, y_train)

# prediction
yhat_lr = lr.predict(x_test)

# performance
lr_result = ml_error('Linear Regression', np.expm1(y_test), np.expm1(yhat_lr))
lr_result #.RMSE[0] #2744.045828082333
# if lr_result.RMSE[0]> baseline_result.RMSE[0] os dados tem comportamento complexo, não linear

In [ ]:
# o erro do modelo linear regression(2744.045828) é maior que o erro do modelo de média(1843.604875)

## 7.3. Linear Regression Regularized Model

In [ ]:
# model
lrr = Lasso(alpha=0.01).fit(x_train, y_train)

# prediction
yhat_lrr = lrr.predict(x_test)

# performance
lrr_result = ml_error('Linear Regression - Lasso', np.expm1(y_test), np.expm1(yhat_lrr))
lrr_result #.RMSE[0] #2744.045828082333
# if lr_result.RMSE[0]> baseline_result.RMSE[0] os dados tem comportamento complexo, sendo pior que da regressão linear normal
# por tando dos 3 modelos o da média mostrou-se melhor com erro de RMSE menor (1843.604875) que os restantes dois

## 7.4. Random Forest Regressor

In [ ]:
#import
from sklearn.ensemble import RandomForestRegressor

# model
rfr = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42).fit(x_train, y_train)

# prediction
yhat_rfr = rfr.predict(x_test)

# performance
rfr_result = ml_error('Ramdom Forest Regressor', np.expm1(y_test), np.expm1(yhat_rfr))
rfr_result #.RMSE[0] #2744.045828082333
# if lr_result.RMSE[0]> baseline_result.RMSE[0] os dados tem comportamento complexo, sendo pior que da regressão linear normal
# por tando dos 3 modelos o da média mostrou-se melhor com erro de RMSE menor (1843.604875) que os restantes dois

## 7.5. XGBoost Regressor

In [ ]:
#/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:54:18] WARNING: /workspace/src/learner.cc:742:
#Parameters: { "colsample_bytee" } are not used.
#  warnings.warn(smsg, UserWarning)

#import
import xgboost as xgb

# model
model_xgbr = xgb.XGBRegressor(
            objective='reg:squarederror',
            n_estimators=100,
            eta=0.01,
            max_depth=10,
            subsample=0.7,
            colsample_bytee=0.9).fit(x_train, y_train)

# prediction
yhat_xgbr = model_xgbr.predict(x_test)

# performance
rgbr_result = ml_error('XGBoost Regressor', np.expm1(y_test), np.expm1(yhat_xgbr))
rgbr_result #.RMSE[0] #2744.045828082333
# if lr_result.RMSE[0]> baseline_result.RMSE[0] os dados tem comportamento complexo, sendo pior que da regressão linear normal
# por tando dos 3 modelos o da média mostrou-se melhor com erro de RMSE menor (1843.604875) que os restantes dois

## 7.6. Compare Model's Performance

In [ ]:

modelling_result = pd.concat([baseline_result, lr_result, lrr_result, rfr_result, rgbr_result])
modelling_result.sort_values('RMSE')

In [ ]:

#feature to add
#feature_to_add = ['date','sales']
cols_selected_boruta_full=[]
cols_selected_boruta_full=cols_selected_boruta.extend(feature_to_add)
x_training = df6_train[cols_selected_boruta]##cols_selected_boruta_full] #.drop(['date','sales'], axis=1).values


In [ ]:
print(cols_selected_boruta) #_full)#] cols_selected_boruta_full) #None

In [ ]:
x_training.sample()

In [ ]:
x_training = df6[cols_selected_boruta] #X_train[cols_selected_boruta]
x_training.shape#()

In [ ]:

  #print('{MAE CV:' + np.round( np.mean(mae_list), 2).astype(str),
  #                      'MAPE CV:'+ np.round( np.mean(mape_list), 2).astype(str) )

#a = cross_validation(x_training, 6)
#(x_training, kfold):
#try_go()
model=LinearRegression()
#criar uma lista e um ciclo for

model_name='Linear Regression'
model = Lasso(alpha=0.01)#.fit(x_train, y_train)

# prediction
#yhat_lrr = lrr.predict(x_test)

# performance
#lrr_result = ml_error('Linear Regression - Lasso', np.expm1(y_test), np.expm1(yhat_lrr))
model_name='Linear Regression - Lasso'
## performance
#rgbr_result = ml_error('XGBoost Regressor', np.expm1(y_test), np.expm1(yhat_xgbr))
model_name = 'XGBoost Regressor'
model = xgb.XGBRegressor(
            objective='reg:squarederror',
            n_estimators=100,
            eta=0.01,
            max_depth=10,
            subsample=0.7,
            colsample_bytee=0.9) #.fit(x_train, y_train)
# model
#rfr = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42).fit(x_train, y_train)

# prediction
#yhat_rfr = rfr.predict(x_test)

# performance
#rfr_result = ml_error('Ramdom Forest Regressor', np.expm1(y_test), np.expm1(yhat_rfr))

model_name='Ramdom Forest Regressor'
model = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)

a = cross_validation(x_training, 6, model_name, model)
a

# 8.0. PASSO 08 - HYPERPARAMETER FINE TUNING

## 8.1. Random Search

In [ ]:
import random
import warnings
warnings.filterwarnings('ignore')
param = {
    'n_estimators'    : [1500, 1700, 2500, 3000, 3500],
              'eta'   : [0.01, 0.03],
          'max_depth' : [3, 5, 9],
          'subsample' : [0.1, 0.5, 0.7],
    'colsample_bytee' : [0.3, 0.7, 0.9],
  'min_child_weight'  : [3,8,15]}

MAX_EVAL = 2


In [ ]:
#import
#import xgboost as xgb
for i in range(MAX_EVAL):
  #choose values for parameters randomly
  hp = { k: random.sample(v,1)[0] for k,v in param.items()}
  print(hp)
  # model
  model_xgbr = xgb.XGBRegressor(
              objective='reg:squarederror',
              n_estimators=hp['n_estimators'],
              eta=hp['eta'],
              max_depth=hp['max_depth'],
              subsample=hp['subsample'],
              colsample_bytee=hp['colsample_bytee'],
              min_child_weight=hp['min_child_weight']) #.fit(x_train, y_train)

  # prediction
  yhat_xgbr = model_xgbr.predict(x_test)

  # performance
  rgbr_result = ml_error('XGBoost Regressor', np.expm1(y_test), np.expm1(yhat_xgbr))
  rgbr_result #.RMSE[0] #2744.045828082333


## 8.2. Final model